# ADHAN Gemma 3 1B-it Training - LoRA Fine-tuning

## Overview
This notebook fine-tunes Google's Gemma 3 1B Instruct model using LoRA (Low-Rank Adaptation)
on ADHAN's modern Tamil corpus with **sequential curriculum learning**.

**Training Strategy**:
- **Model**: `google/gemma-3-1b-it` (1B parameters, instruction-tuned)
- **Method**: 4-bit LoRA fine-tuning (QLoRA)
- **Corpus**: Modern Tamil Enhanced (train/val/test splits from `data/final/tamil_texts/hf/`)
- **Learning**: Curriculum learning — **Phase 1** (modern sources) → **Phase 2** (classical sources)
- **Framework**: Hugging Face Transformers + PEFT

**Benefits over XLM-RoBERTa**:
1. Better Tamil Understanding: Gemma 3 has 2T token pretraining including Tamil
2. Instruction-Following: Built-in chat format for Q&A tasks
3. Mobile Deployment: Converts to GGUF (~600MB) for on-device inference
4. Faster Training: 4-bit LoRA training completes in 30-45 minutes on single GPU

---

## 1. Setup & Configuration

In [ ]:
import os
import json
from pathlib import Path
from datetime import datetime
import numpy as np
import torch
import transformers

from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    BitsAndBytesConfig,
    TrainingArguments,
    Trainer,
    DataCollatorForLanguageModeling,
)
from peft import LoraConfig, get_peft_model, TaskType, PeftModel
from datasets import load_dataset, Dataset

# Reproducibility
torch.manual_seed(42)
np.random.seed(42)
os.environ['PYTHONHASHSEED'] = '42'

# ── Paths ──────────────────────────────────────────────────────────────────
PROJECT_ROOT = Path(os.environ.get('ADHAN_PROJECT_ROOT', Path.cwd().parent.parent))
DATA_DIR     = PROJECT_ROOT / 'data' / 'final' / 'tamil_texts' / 'hf'
MODELS_DIR   = PROJECT_ROOT / 'models' / 'adhan-gemma-v1'
ADAPTER_DIR  = MODELS_DIR / 'lora_adapter'
CKPT_DIR     = PROJECT_ROOT / 'models' / 'checkpoints' / 'gemma'

for d in [MODELS_DIR, ADAPTER_DIR, CKPT_DIR]:
    d.mkdir(parents=True, exist_ok=True)

# ── Key configuration ──────────────────────────────────────────────────────
MODEL_NAME    = 'google/gemma-3-1b-it'
MAX_LENGTH    = 512
LORA_R        = 16
LORA_ALPHA    = 32
LORA_DROPOUT  = 0.05
EPOCHS        = 3
BATCH_SIZE    = 4
LEARNING_RATE = 2e-4

print(f'PyTorch Version:       {torch.__version__}')
print(f'Transformers Version:  {transformers.__version__}')
print(f'CUDA Available:        {torch.cuda.is_available()}')
if torch.cuda.is_available():
    print(f'GPU Device:            {torch.cuda.get_device_name(0)}')
    print(f'GPU Memory:            {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB')
print(f'\nProject root:  {PROJECT_ROOT}')
print(f'Data dir:      {DATA_DIR}')
print(f'Output dir:    {MODELS_DIR}')

## 2. Data Loading & Tokenization

In [ ]:
def load_jsonl(path: Path) -> list[dict]:
    """Load a JSONL file and return a list of dicts."""
    records = []
    with open(path, 'r', encoding='utf-8') as f:
        for line in f:
            line = line.strip()
            if line:
                records.append(json.loads(line))
    return records


def resolve_val_path(data_dir: Path) -> Path:
    """Support both val.jsonl and validation.jsonl naming conventions."""
    for name in ('val.jsonl', 'validation.jsonl'):
        p = data_dir / name
        if p.exists():
            return p
    raise FileNotFoundError(f'No val/validation JSONL found in {data_dir}')


# Source classification for curriculum learning
MODERN_SOURCES    = {'social', 'news', 'modern_conversational', 'wikipedia', 'web'}
CLASSICAL_SOURCES = {'thirukkural', 'sangam', 'classical', 'classical_poetry',
                     'purananuru', 'akananuru', 'natrinai', 'kurunthokai'}


def classify_source(record: dict) -> str:
    """Return 'modern', 'classical', or 'other' for a data record."""
    src = record.get('source', '').lower()
    if src in MODERN_SOURCES:
        return 'modern'
    if src in CLASSICAL_SOURCES:
        return 'classical'
    return 'other'


print('Loading JSONL splits...')
train_records = load_jsonl(DATA_DIR / 'train.jsonl')
val_records   = load_jsonl(resolve_val_path(DATA_DIR))
test_records  = load_jsonl(DATA_DIR / 'test.jsonl')

print(f'  Train:      {len(train_records):,} records')
print(f'  Validation: {len(val_records):,} records')
print(f'  Test:       {len(test_records):,} records')

# Split training records by curriculum phase
modern_records    = [r for r in train_records if classify_source(r) == 'modern']
classical_records = [r for r in train_records if classify_source(r) == 'classical']
other_records     = [r for r in train_records if classify_source(r) == 'other']

# Records not explicitly classical participate in Phase 1
phase1_records = modern_records + other_records

# Phase 2 uses classical records; if none found, fall back to unclassified
# (other_records) to avoid retraining on modern sources a second time.
if classical_records:
    phase2_records = classical_records
    phase2_label   = 'classical'
elif other_records:
    import warnings
    warnings.warn('No classical records found — Phase 2 will use unclassified records.')
    phase2_records = other_records
    phase2_label   = 'other (fallback — no classical found)'
else:
    import warnings
    warnings.warn('No classical or unclassified records — Phase 2 will reuse full training set.')
    phase2_records = train_records
    phase2_label   = 'full training set (fallback)'

print(f'\nCurriculum split:')
print(f'  Phase 1 (modern + other):  {len(phase1_records):,} records')
print(f'  Phase 2 ({phase2_label}): {len(phase2_records):,} records')
print(f'\nSample keys: {list(train_records[0].keys())}')
print(f'Sample text (first 200 chars):\n  {train_records[0].get("text", "")[:200]}')

In [ ]:
def to_gemma_format(text: str) -> str:
    """
    Wrap plain Tamil text in Gemma 3's instruction-following chat template.

    The model is prompted to continue / explain the passage so that the
    fine-tuning signal teaches it about Tamil language and culture.
    """
    return (
        '<start_of_turn>user\n'
        'தமிழ் உரையை தொடர்ந்து எழுதவும்:\n'
        f'{text}<end_of_turn>\n'
        '<start_of_turn>model\n'
        f'{text}<end_of_turn>'
    )


def records_to_dataset(records: list[dict]) -> Dataset:
    """Convert a list of JSONL records to a HuggingFace Dataset."""
    texts = [to_gemma_format(r['text']) for r in records]
    return Dataset.from_dict({'text': texts})


# Full train / val / test for final evaluation
raw_train  = records_to_dataset(train_records)
raw_val    = records_to_dataset(val_records)
raw_test   = records_to_dataset(test_records)

# Phase-specific training sets for curriculum learning
raw_phase1 = records_to_dataset(phase1_records)
raw_phase2 = records_to_dataset(phase2_records)

print(f'Dataset sizes:')
print(f'  Full train:  {len(raw_train):,}')
print(f'  Phase 1:     {len(raw_phase1):,}')
print(f'  Phase 2:     {len(raw_phase2):,}')
print(f'\nSample formatted entry:')
print(raw_train[0]['text'][:400])

In [ ]:
print(f'Loading tokenizer for {MODEL_NAME} ...')
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = 'right'


def tokenize(batch):
    return tokenizer(
        batch['text'],
        truncation=True,
        max_length=MAX_LENGTH,
        padding='max_length',
    )


def prepare_dataset(raw: Dataset) -> Dataset:
    """Tokenize a raw text dataset and attach causal-LM labels."""
    ds = raw.map(tokenize, batched=True, remove_columns=['text'])
    ds = ds.map(lambda ex: {'labels': ex['input_ids']})
    ds.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'])
    return ds


print('Tokenizing datasets...')
train_dataset  = prepare_dataset(raw_train)
val_dataset    = prepare_dataset(raw_val)
test_dataset   = prepare_dataset(raw_test)
phase1_dataset = prepare_dataset(raw_phase1)
phase2_dataset = prepare_dataset(raw_phase2)

print(f'\n✅ Tokenized datasets:')
print(f'   Train (full): {train_dataset}')
print(f'   Validation:   {val_dataset}')
print(f'   Test:         {test_dataset}')
print(f'   Phase 1:      {phase1_dataset}')
print(f'   Phase 2:      {phase2_dataset}')

## 3. Load Model in 4-bit Quantization & Apply LoRA

In [ ]:
print('Configuring 4-bit quantization (QLoRA)...')
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type='nf4',
    bnb_4bit_compute_dtype=torch.float16,
)

print(f'Loading {MODEL_NAME} ...')
model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    quantization_config=bnb_config,
    device_map='auto',
    torch_dtype=torch.float16,
)
model.config.use_cache = False  # Required for gradient checkpointing

param_count = sum(p.numel() for p in model.parameters())
print(f'\n✅ Model loaded: {param_count / 1e9:.2f}B parameters')

In [ ]:
print('Applying LoRA adapter...')
lora_config = LoraConfig(
    task_type=TaskType.CAUSAL_LM,
    r=LORA_R,
    lora_alpha=LORA_ALPHA,
    lora_dropout=LORA_DROPOUT,
    target_modules=['q_proj', 'v_proj', 'k_proj', 'o_proj'],
    bias='none',
)

model = get_peft_model(model, lora_config)
model.print_trainable_parameters()

## 4. Sequential / Curriculum Training

Training proceeds in two sequential phases:

| Phase | Data | Epochs | Learning Rate | Purpose |
|-------|------|--------|---------------|---------|
| 1 | Modern Tamil (social, news, web, wikipedia) | 2 | 2e-4 | Fluency & modern usage |
| 2 | Classical Tamil (Thirukkural, Sangam, etc.) | 1 | 1e-4 | Classical literature |

In [ ]:
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

PHASE1_EPOCHS = 2
PHASE2_EPOCHS = 1

# ── Phase 1: Modern Tamil — learn fluency & contemporary usage ────────────
phase1_args = TrainingArguments(
    output_dir=str(CKPT_DIR / 'phase1'),
    num_train_epochs=PHASE1_EPOCHS,
    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=BATCH_SIZE,
    learning_rate=LEARNING_RATE,
    fp16=True,
    logging_steps=10,
    eval_strategy='steps',
    eval_steps=100,
    save_steps=100,
    save_total_limit=2,
    load_best_model_at_end=True,
    metric_for_best_model='eval_loss',
    greater_is_better=False,
    warmup_ratio=0.05,
    lr_scheduler_type='cosine',
    gradient_checkpointing=True,
    report_to='none',
    run_name=f'adhan-gemma-phase1-{datetime.now().strftime("%Y%m%d_%H%M")}',
)

phase1_trainer = Trainer(
    model=model,
    args=phase1_args,
    train_dataset=phase1_dataset,
    eval_dataset=val_dataset,
    data_collator=data_collator,
)

print('=' * 70)
print('PHASE 1 — Modern Tamil Sources')
print('=' * 70)
print(f'  Records:       {len(phase1_dataset):,}')
print(f'  Epochs:        {PHASE1_EPOCHS}')
print(f'  Learning rate: {LEARNING_RATE}')

phase1_result = phase1_trainer.train()
print(f'\n✅ Phase 1 complete!')
print(f'   Loss:    {phase1_result.training_loss:.4f}')
print(f'   Runtime: {phase1_result.metrics["train_runtime"]:.0f}s')

# ── Phase 2: Classical Tamil — refine on literary corpus ─────────────────
phase2_args = TrainingArguments(
    output_dir=str(CKPT_DIR / 'phase2'),
    num_train_epochs=PHASE2_EPOCHS,
    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=BATCH_SIZE,
    # Lower LR preserves Phase 1 learning while fine-tuning on classical sources
    learning_rate=LEARNING_RATE / 2,
    fp16=True,
    logging_steps=10,
    eval_strategy='steps',
    eval_steps=100,
    save_steps=100,
    save_total_limit=2,
    load_best_model_at_end=True,
    metric_for_best_model='eval_loss',
    greater_is_better=False,
    warmup_ratio=0.03,
    lr_scheduler_type='cosine',
    gradient_checkpointing=True,
    report_to='none',
    run_name=f'adhan-gemma-phase2-{datetime.now().strftime("%Y%m%d_%H%M")}',
)

phase2_trainer = Trainer(
    model=model,
    args=phase2_args,
    train_dataset=phase2_dataset,
    eval_dataset=val_dataset,
    data_collator=data_collator,
)

print('\n' + '=' * 70)
print('PHASE 2 — Classical Tamil Sources')
print('=' * 70)
print(f'  Records:       {len(phase2_dataset):,}')
print(f'  Epochs:        {PHASE2_EPOCHS}')
print(f'  Learning rate: {LEARNING_RATE / 2}')

phase2_result = phase2_trainer.train()
print(f'\n✅ Phase 2 complete!')
print(f'   Loss:    {phase2_result.training_loss:.4f}')
print(f'   Runtime: {phase2_result.metrics["train_runtime"]:.0f}s')

# Expose the final trainer for downstream evaluation / saving
trainer      = phase2_trainer
train_result = phase2_result

## 5. Evaluation

In [ ]:
print('Evaluating on validation set...')
val_metrics = trainer.evaluate(eval_dataset=val_dataset)
print(f'  Validation loss:     {val_metrics["eval_loss"]:.4f}')

print('\nEvaluating on test set...')
test_metrics = trainer.evaluate(eval_dataset=test_dataset)
print(f'  Test loss:           {test_metrics["eval_loss"]:.4f}')

In [ ]:
def generate_tamil(prompt: str, max_new_tokens: int = 200) -> str:
    """Generate Tamil text from a given prompt using the fine-tuned model."""
    formatted = (
        '<start_of_turn>user\n'
        f'{prompt}<end_of_turn>\n'
        '<start_of_turn>model\n'
    )
    inputs = tokenizer(formatted, return_tensors='pt').to(model.device)
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            temperature=0.7,
            do_sample=True,
            top_p=0.9,
            repetition_penalty=1.1,
            pad_token_id=tokenizer.eos_token_id,
        )
    decoded = tokenizer.decode(outputs[0], skip_special_tokens=True)
    # Return only the model's response
    if 'model\n' in decoded:
        return decoded.split('model\n', 1)[-1].strip()
    return decoded.strip()


test_prompts = [
    'திருக்குறளில் உள்ள ஒரு குறள் சொல்லவும்',
    'தமிழ் இலக்கியத்தின் சிறப்புகள் யாவை?',
    'சங்க இலக்கியம் பற்றி சுருக்கமாக விளக்கவும்',
]

for prompt in test_prompts:
    print(f'\n📝 Prompt: {prompt}')
    response = generate_tamil(prompt)
    print(f'💬 Response: {response[:300]}')

    # Check Tamil character ratio
    tamil_chars = sum(1 for c in response if '\u0B80' <= c <= '\u0BFF')
    total_chars = max(sum(1 for c in response if c.strip()), 1)
    tamil_ratio = tamil_chars / total_chars
    print(f'📊 Tamil character ratio: {tamil_ratio:.1%}')

## 6. Save Model & Adapter

In [ ]:
print(f'Saving LoRA adapter to {ADAPTER_DIR} ...')
model.save_pretrained(str(ADAPTER_DIR))

print(f'Saving tokenizer to {MODELS_DIR} ...')
tokenizer.save_pretrained(str(MODELS_DIR))

# Persist training configuration
config = {
    'model_name': MODEL_NAME,
    'max_length': MAX_LENGTH,
    'lora_r': LORA_R,
    'lora_alpha': LORA_ALPHA,
    'lora_dropout': LORA_DROPOUT,
    'epochs': EPOCHS,
    'batch_size': BATCH_SIZE,
    'learning_rate': LEARNING_RATE,
    'train_loss': train_result.training_loss,
    'val_loss': val_metrics['eval_loss'],
    'test_loss': test_metrics['eval_loss'],
    'timestamp': datetime.now().isoformat(),
}
with open(MODELS_DIR / 'training_config.json', 'w', encoding='utf-8') as f:
    json.dump(config, f, indent=2, ensure_ascii=False)

print('\n✅ Saved:')
print(f'   LoRA adapter: {ADAPTER_DIR}')
print(f'   Tokenizer:    {MODELS_DIR}')
print(f'   Config:       {MODELS_DIR / "training_config.json"}')

## 7. GGUF Conversion Instructions

After training, merge the LoRA adapter into the base model and convert to GGUF format
for efficient on-device inference using llama.cpp.

### Step 1 – Merge LoRA adapter into the base model

```python
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import PeftModel
import torch

base_model = AutoModelForCausalLM.from_pretrained(
    'google/gemma-3-1b-it',
    torch_dtype=torch.float16,
    device_map='auto',
)
merged_model = PeftModel.from_pretrained(base_model, 'models/adhan-gemma-v1/lora_adapter')
merged_model = merged_model.merge_and_unload()
merged_model.save_pretrained('models/adhan-gemma-v1/merged')

tokenizer = AutoTokenizer.from_pretrained('models/adhan-gemma-v1')
tokenizer.save_pretrained('models/adhan-gemma-v1/merged')
```

### Step 2 – Clone llama.cpp and install dependencies

```bash
git clone https://github.com/ggerganov/llama.cpp
cd llama.cpp
pip install -r requirements.txt
```

### Step 3 – Convert to GGUF

```bash
python convert_hf_to_gguf.py \
    ../models/adhan-gemma-v1/merged \
    --outfile ../models/adhan-gemma-v1/adhan-gemma-q4_k_m.gguf \
    --outtype q4_k_m
```

This produces a **~600 MB** Q4_K_M quantized GGUF file suitable for:
- Desktop inference via `llama.cpp`
- Mobile deployment via `llama.rn` or `MLC LLM`
- Server-side inference with `Ollama`

### Step 4 – Test with llama.cpp

```bash
cd llama.cpp
make -j$(nproc)
./main \
    -m ../models/adhan-gemma-v1/adhan-gemma-q4_k_m.gguf \
    -p '<start_of_turn>user\nதிருக்குறளில் உள்ள ஒரு குறள் சொல்லவும்<end_of_turn>\n<start_of_turn>model\n' \
    -n 200
```